In [1]:
import os
import cv2
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [1]:
import os
import cv2
from datetime import datetime


def extract_date_from_filename(filename):
    """
    Extracts the date from a filename based on a specific format.

    Assumes the filename format is 'S2A_MSIL1C_YYYYMMDD...tif', where the date 
    is embedded as the third part of the filename separated by underscores.

    Parameters:
    - filename: The filename from which to extract the date.

    Returns:
    - A datetime object representing the extracted date.
    """
    # Split the filename by underscore and extract the third element,
    # which is expected to be the date string in 'YYYYMMDDTHHMMSS' format.
    date_str = filename.split('_')[2]

    # Convert the date string into a datetime object and return it.
    return datetime.strptime(date_str, '%Y%m%dT%H%M%S')

def load_and_crop_images(directories, filenames, crop_size=(128, 128), save_dir_gt='/home/jovyan/work/saved_data/HAT/datasets/SI2gt', save_dir_lq='/home/jovyan/work/saved_data/HAT/datasets/SI2lq'):
    """
    Loads images from multiple directories, crops them into smaller sections,
    and saves the cropped images in a specified directory.

    Parameters:
    - directories: A list of directories to search for images.
    - filenames: A list of filenames to load and crop.
    - crop_size: The size of each crop (width, height).
    - save_dir: Directory where the cropped images will be saved.
    """
    if not os.path.exists(save_dir_gt):
        os.makedirs(save_dir_gt)
    if not os.path.exists(save_dir_lq):
        os.makedirs(save_dir_lq)

    for directory in directories:
        for filename in filenames:
            file_path = os.path.join(directory, filename)
            image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)

            if image is not None:
                # Extract the date from the filename
                date = extract_date_from_filename(filename)

                # Crop the image into smaller sections
                for i in range(0, image.shape[0], crop_size[1]):
                    for j in range(0, image.shape[1], crop_size[0]):
                        cropped_image_gt = image[i:i+crop_size[1], j:j+crop_size[0]]
                        crop_index = (i // crop_size[1], j // crop_size[0])

                        # Generate a unique filename for each cropped gt image
                        cropped_filename_gt = f"{filename}_crop_{crop_index[0]}_{crop_index[1]}_gt.png"
                        save_path_gt = os.path.join(save_dir_gt, cropped_filename_gt)

                        # Save the cropped gt image
                        cv2.imwrite(save_path_gt, cropped_image_gt)
                        
                        #Downsample the cropped image
                        new_size = (crop_size[0] // 2, crop_size[1] // 2)
                        cropped_image_lq = cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)
                        
                        # Generate a unique filename for each cropped lq image
                        cropped_filename_lq = f"{filename}_crop_{crop_index[0]}_{crop_index[1]}_lq.png"
                        save_path_lq = os.path.join(save_dir_lq, cropped_filename_lq)

                        # Save the cropped lq image
                        cv2.imwrite(save_path_lq, cropped_image_lq)
                        
            else:
                print(f"Failed to load image: {file_path}")


In [25]:
def filter_images_by_tile(directories, tile_code):
    """
    Collects a list of filenames from multiple directories that contain 
    a specific MGRS tile code.

    Parameters:
    - directories: A list of directories containing the image files.
    - tile_code: The MGRS tile code to filter by (e.g., 'T32UPB').

    Returns:
    - A list of filenames that contain the specified MGRS tile code.
    """
    # List to hold the filenames that match the tile_code
    filtered_filenames = []

    # Iterate over each directory
    for directory in directories:
        # Iterate over all files in the given directory
        for filename in os.listdir(directory):
            # Check if the tile_code is part of the filename
            if tile_code in filename:
                filtered_filenames.append(filename)

    return filtered_filenames

In [ ]:
directories = ['/home/jovyan/work/satellite_data/v1', '/home/jovyan/work/satellite_data/v2']
specific_tile_code = 'T33UWS'

#Set image size
#img_height = 128
#img_width = 128

# Collecting image names that match the specific tile code
img_names = filter_images_by_tile(directories, specific_tile_code)

# Load and resize images, and also get their dates
load_and_crop_images2(directories, img_names)

In [7]:
import os

def count_images_in_directory(directory):
    """
    Zählt die Anzahl der Bilddateien in einem Verzeichnis.

    Parameter:
    - directory: Der Pfad zum Verzeichnis.

    Gibt zurück:
    - Die Anzahl der Bilddateien im Verzeichnis.
    """
    # Unterstützte Bildformate
    supported_formats = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}
    
    # Zählen der Bilddateien
    image_count = sum(1 for file in os.listdir(directory) if os.path.splitext(file)[1].lower() in supported_formats)
    
    return image_count

# Beispielaufruf der Funktion
directory = '/home/jovyan/work/saved_data/HAT/datasets/SI2gt'
print(f"Anzahl der Bilder im Verzeichnis '{directory}': {count_images_in_directory(directory)}")


Anzahl der Bilder im Verzeichnis '/home/jovyan/work/saved_data/HAT/datasets/SI2gt': 59168


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

def plot_top_left_images(directory):
    """
    Plottet alle Bilder aus der linken oberen Ecke eines Verzeichnisses.

    Parameter:
    - directory: Der Pfad zum Verzeichnis mit den Bildern.
    """
    # Iteriere durch alle Dateien im Verzeichnis
    for filename in os.listdir(directory):
        # Prüfe, ob die Datei ein Bild aus der linken oberen Ecke ist
        if 'crop_0_0' in filename:
            # Lade und plotte das Bild
            img_path = os.path.join(directory, filename)
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Konvertiere von BGR zu RGB

            plt.figure()
            plt.imshow(image)
            plt.title(filename)
            plt.axis('off')
            plt.show()

# Beispielaufruf der Funktion
directory = '/home/jovyan/work/saved_data/training_data'
plot_top_left_images(directory)


In [38]:
def load_and_crop_images2(directories, filenames, crop_size=(128, 128), save_dir_gt='/home/jovyan/work/saved_data/HAT/datasets/SI2gt', save_dir_lq='/home/jovyan/work/saved_data/HAT/datasets/SI2lq'):
    """
    Loads images from specified directories, crops them into smaller sections
    according to the given crop size, and saves the cropped images into separate 
    directories for ground truth (high-resolution) and low-resolution images.

    Parameters:
    - directories: A list of directories where the images are located.
    - filenames: A list of filenames of the images to be loaded and cropped.
    - crop_size: A tuple defining the size of each crop (width, height).
    - save_dir_gt: Directory where the cropped high-resolution images will be saved.
    - save_dir_lq: Directory where the corresponding low-resolution images will be saved.
    """
    # Create the directories if they don't exist
    if not os.path.exists(save_dir_gt):
        os.makedirs(save_dir_gt)
    if not os.path.exists(save_dir_lq):
        os.makedirs(save_dir_lq)

    for directory in directories:
        for filename in filenames:
            file_path = os.path.join(directory, filename)
            image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)

            if image is not None:

                # Crop the image into smaller sections based on the specified crop size
                for i in range(0, image.shape[0], crop_size[1]):
                    for j in range(0, image.shape[1], crop_size[0]):
                        # Get the cropped high-resolution image
                        cropped_image_gt = image[i:i+crop_size[1], j:j+crop_size[0]]
                        crop_index = (i // crop_size[1], j // crop_size[0])
                        
                        #Check if crop has the correct size
                        if cropped_image_gt.shape[0:2] == crop_size:

                            #Remove .tif from filename
                            filename_with_extension = filename
                            filename_without_extension, _ = os.path.splitext(filename_with_extension)

                            # Create a unique filename for each cropped high-resolution image
                            cropped_filename_gt = f"{filename_without_extension}_crop_{crop_index[0]}_{crop_index[1]}.png"
                            save_path_gt = os.path.join(save_dir_gt, cropped_filename_gt)

                            # Save the cropped high-resolution image
                            cv2.imwrite(save_path_gt, cropped_image_gt)

                            # Downsample the cropped high-resolution image to create a low-resolution version
                            new_size = (crop_size[0] // 2, crop_size[1] // 2)
                            cropped_image_lq = cv2.resize(cropped_image_gt, new_size, interpolation=cv2.INTER_AREA)

                            # Create a unique filename for each cropped low-resolution image
                            cropped_filename_lq = f"{filename_without_extension}_crop_{crop_index[0]}_{crop_index[1]}.png"
                            save_path_lq = os.path.join(save_dir_lq, cropped_filename_lq)

                            # Save the cropped low-resolution image
                            cv2.imwrite(save_path_lq, cropped_image_lq)
                            
                        else:
                            print(f"Image {filename} crop {i//crop_size[1]}_{j//crop_size[0]} does not match the required size {crop_size}")
                        
            else:
                # Print an error message if the image could not be loaded
                print(f"Failed to load image: {file_path}")


In [ ]:
import os

def list_first_10_filenames(directory):
    """
    Listet die ersten 10 Dateinamen im angegebenen Verzeichnis auf.

    :param directory: Pfad zum Verzeichnis, dessen Dateinamen aufgelistet werden sollen.
    """
    try:
        # Auflisten aller Dateien im angegebenen Verzeichnis
        filenames = os.listdir(directory)

        # Beschränkung auf die ersten 10 Dateien
        first_10_files = filenames[:10]

        # Ausgabe der Dateinamen
        for file in first_10_files:
            print(file)
    except FileNotFoundError:
        print(f"Verzeichnis nicht gefunden: {directory}")
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

# Verwendung der Funktion - Ersetzen Sie 'IhrVerzeichnispfad' mit dem tatsächlichen Pfad
directory_path = '/home/jovyan/work/saved_data/HAT/datasets/SI2gt'
list_first_10_filenames(directory_path)


In [7]:
from datasets import load_dataset
dataset = load_dataset('eugenesiow/Set5', 'bicubic_x3', split='validation')

type(dataset)

datasets.arrow_dataset.Dataset

In [20]:
print(dataset[0]["hr"])
mpimg.imread(dataset[0]["hr"])

/storage/hf-datasets-cache/all/datasets/61269473884020-config-parquet-and-info-eugenesiow-Set5-64a92ea9/downloads/extracted/3ac517a00f2bce893d0cfec525d48773fb91f3aac172dc518f51d75e9beae106/Set5_HR/baby.png


FileNotFoundError: [Errno 2] No such file or directory: '/storage/hf-datasets-cache/all/datasets/61269473884020-config-parquet-and-info-eugenesiow-Set5-64a92ea9/downloads/extracted/3ac517a00f2bce893d0cfec525d48773fb91f3aac172dc518f51d75e9beae106/Set5_HR/baby.png'

In [14]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# Laden Sie Ihren Datensatz
from datasets import load_dataset
dataset = load_dataset('eugenesiow/Set5', 'bicubic_x2', split='validation')

# Anzahl der anzuzeigenden und zu speichernden Bildpaare
num_pairs_to_display = 5

# Verzeichnisse für HR- und LR-Bilder
hr_directory = '/home/jovyan/work/saved_data/HAT/datasets/Set5/GTmod3'
lr_directory = '/home/jovyan/work/saved_data/HAT/datasets/Set5/LRbicx3'

# Stellen Sie sicher, dass die Verzeichnisse existieren
os.makedirs(hr_directory, exist_ok=True)
os.makedirs(lr_directory, exist_ok=True)

# Bildpaare speichern und anzeigen
for i in range(min(num_pairs_to_display, len(dataset))):
    # HR- und LR-Bilder extrahieren
    hr_image_path = dataset[i]['hr']
    lr_image_path = dataset[i]['lr']
    print(hr_image_path)

    # HR- und LR-Bilder laden
    hr_image = mpimg.imread(hr_image_path)
    lr_image = mpimg.imread(lr_image_path)

    # HR- und LR-Bilder speichern
    hr_image_save_path = os.path.join(hr_directory, f'image_{i}.png')
    lr_image_save_path = os.path.join(lr_directory, f'image_{i}.png')
    mpimg.imsave(hr_image_save_path, hr_image)
    mpimg.imsave(lr_image_save_path, lr_image)

    # Plotten (optional, falls Sie die Bilder auch anzeigen möchten)
    plt.figure(figsize=(12, 6))

    # HR-Bild
    plt.subplot(1, 2, 1)
    plt.imshow(hr_image)
    plt.title(f"HR-Bild {i}")
    plt.axis('off')

    # LR-Bild
    plt.subplot(1, 2, 2)
    plt.imshow(lr_image)
    plt.title(f"LR-Bild {i}")
    plt.axis('off')

    plt.show()


/storage/hf-datasets-cache/all/datasets/61269473884020-config-parquet-and-info-eugenesiow-Set5-64a92ea9/downloads/extracted/3ac517a00f2bce893d0cfec525d48773fb91f3aac172dc518f51d75e9beae106/Set5_HR/baby.png


FileNotFoundError: [Errno 2] No such file or directory: '/storage/hf-datasets-cache/all/datasets/61269473884020-config-parquet-and-info-eugenesiow-Set5-64a92ea9/downloads/extracted/3ac517a00f2bce893d0cfec525d48773fb91f3aac172dc518f51d75e9beae106/Set5_HR/baby.png'

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Laden Sie Ihren Datensatz
from datasets import load_dataset
dataset = load_dataset('eugenesiow/Set5', 'bicubic_x3', split='validation')

# Anzahl der anzuzeigenden Bildpaare
num_pairs_to_display = 5

# Bildpaare anzeigen
for i in range(min(num_pairs_to_display, len(dataset))):
    # HR- und LR-Bilder extrahieren
    hr_image_path = dataset[i]['hr']
    lr_image_path = dataset[i]['lr']

    # HR- und LR-Bilder laden
    hr_image = mpimg.imread(hr_image_path)
    lr_image = mpimg.imread(lr_image_path)

    # Plotten
    plt.figure(figsize=(12, 6))

    # HR-Bild
    plt.subplot(1, 2, 1)
    plt.imshow(hr_image)
    plt.title(f"HR-Bild {i}")
    plt.axis('off')

    # LR-Bild
    plt.subplot(1, 2, 2)
    plt.imshow(lr_image)
    plt.title(f"LR-Bild {i}")
    plt.axis('off')

    plt.show()


Generating validation split:   0%|          | 0/5 [00:00<?, ? examples/s]

FileNotFoundError: [Errno 2] No such file or directory: '/storage/hf-datasets-cache/all/datasets/61269473884020-config-parquet-and-info-eugenesiow-Set5-64a92ea9/downloads/extracted/3ac517a00f2bce893d0cfec525d48773fb91f3aac172dc518f51d75e9beae106/Set5_HR/baby.png'

In [ ]:
import os
import cv2
from matplotlib import pyplot as plt
from satbm_utils import calculate_brightness
 
def show_first_10_images_original_size(directory_path):
    """
    Displays the first 10 images in a given directory at their original size.

    Parameters:
    - directory_path: Path to the directory containing images.
    """
    # List all files in the given directory and filter out non-image files
    image_files = [file for file in os.listdir(directory_path) if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

    # Display the first 10 images
    for i, image_file in enumerate(image_files[:20]):
        # Read the image from the file
        image_path = os.path.join(directory_path, image_file)
        image = cv2.imread(image_path)
        #print(calculate_brightness(image))
        
        # Get the image dimensions (height, width)
        height, width = image.shape[:2]
        
        # Create a new figure with original image dimensions
        plt.figure(figsize=(width / 100, height / 100))
        
        # Convert the image from BGR (OpenCV default) to RGB for correct color display
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        # Turn off the axis for a cleaner display
        plt.axis('off')
        
        # Display the image
        plt.show()

# This function is intended to be used with a valid directory path
show_first_10_images_original_size("/home/jovyan/work/saved_data/HAT/datasets/SI3gt")






In [1]:
import os

def count_images_in_directories(directories):
    """
    Count the number of image files in a list of directories.

    Parameters:
    - directories: List of strings, paths to the directories to be counted.

    Returns:
    - A list of integers representing the number of image files in each directory.
    """
    # Supported image file extensions
    image_extensions = ('.jpeg', '.jpg', '.png', '.gif', '.bmp', '.tif')

    # Initialize a list to store the count of images for each directory
    image_counts = []

    # Loop through each directory and count the number of image files
    for directory in directories:
        # Check if directory exists
        if not os.path.exists(directory):
            print(f"Directory not found: {directory}")
            image_counts.append(0)
            continue

        image_count = len([file for file in os.listdir(directory) if file.lower().endswith(image_extensions)])
        image_counts.append(image_count)

    return image_counts

# Example usage
# Replace the paths with paths to your actual directories
directories = ['/home/jovyan/work/saved_data/HAT/datasets/SI3gt', '/home/jovyan/work/satellite_data/v2']
image_counts = count_images_in_directories(directories)
print(f"Number of images in each directory: {image_counts}")


Number of images in each directory: [466707, 1150]


In [12]:
from PIL import Image

def crop_top_left_corner(image_path, output_path, crop_width, crop_height):
    """
    Crops the top left corner of an image and saves the cropped image.

    Parameters:
    - image_path: Path to the input image.
    - output_path: Path to save the cropped image.
    - crop_width: The width of the crop area.
    - crop_height: The height of the crop area.

    Returns:
    - None
    """
    with Image.open(image_path) as img:
        # The crop rectangle, as a (left, upper, right, lower)-tuple.
        crop_rectangle = (0, 0, crop_width, crop_height)
        cropped_img = img.crop(crop_rectangle)
        cropped_img.save(output_path)

# Example usage:
crop_top_left_corner('/home/jovyan/work/saved_data/img/test.png', '/home/jovyan/work/saved_data/img/cropped_image_t.jpg', 1024, 1024)


In [17]:
import os
import cv2
from datetime import datetime
import numpy as np

def load_and_crop_images(directories, scale, crop_size=(540, 540), save_dir_gt='/home/jovyan/work/saved_data/HAT/datasets/SI3gt', save_dir_lq='/home/jovyan/work/saved_data/HAT/datasets/SI3lq'):
    """
    Loads and crops all images from multiple directories, then saves them.
    
    Parameters:
    - directories: A list of directories to search for images.
    - crop_size: The size of each crop (width, height).
    - save_dir_gt: Directory to save the cropped high quality images.
    - save_dir_lq: Directory to save the cropped low quality images.
    """
    if not os.path.exists(save_dir_gt):
        os.makedirs(save_dir_gt)
    if not os.path.exists(save_dir_lq):
        os.makedirs(save_dir_lq)

    for directory in directories:
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)

            if image is not None:
                # Crop the image into smaller sections
                for i in range(0, image.shape[0], crop_size[1]):
                    for j in range(0, image.shape[1], crop_size[0]):
                        cropped_image_gt = image[i:i+crop_size[1], j:j+crop_size[0]]
                        crop_index = (i // crop_size[1], j // crop_size[0])
                        
                        if np.sum(cropped_image_gt == 0) == 0:  # Prüfen auf nicht-schwarze Pixel
                            # Überprüfen, ob die Kachel die volle Größe hat (für Kantenkacheln)
                            if cropped_image_gt.shape[0] == crop_size[0] and cropped_image_gt.shape[1] == crop_size[1]:

                                # Generate a unique filename for each cropped gt image
                                cropped_filename_gt = f"{filename}_crop_{crop_index[0]}_{crop_index[1]}.png"
                                save_path_gt = os.path.join(save_dir_gt, cropped_filename_gt)

                                # Save the cropped gt image
                                cv2.imwrite(save_path_gt, cropped_image_gt)
                        
                                # Downsample the cropped image
                                new_size = (crop_size[0] // scale, crop_size[1] // scale)
                                cropped_image_lq = cv2.resize(cropped_image_gt, new_size, interpolation=cv2.INTER_AREA)
                        
                                # Generate a unique filename for each cropped lq image
                                cropped_filename_lq = f"{filename}_crop_{crop_index[0]}_{crop_index[1]}.png"
                                save_path_lq = os.path.join(save_dir_lq, cropped_filename_lq)

                                # Save the cropped lq image
                                cv2.imwrite(save_path_lq, cropped_image_lq)
                                
directories = ['/home/jovyan/work/satellite_data/v1', '/home/jovyan/work/satellite_data/v2']
load_and_crop_images(directories, 3)


In [12]:
import os
import cv2
from datetime import datetime
import numpy as np

def load_and_crop_images(path, scale, crop_size=(5490, 5490), save_dir_gt='/home/jovyan/work/saved_data/simg-benchmark/SI2gt', save_dir_lq='/home/jovyan/work/saved_data/simg-benchmark/SI2lq'):
    """
    Loads and crops all images from multiple directories, then saves them.
    
    Parameters:
    - directories: A list of directories to search for images.
    - crop_size: The size of each crop (width, height).
    - save_dir_gt: Directory to save the cropped high quality images.
    - save_dir_lq: Directory to save the cropped low quality images.
    """
    if not os.path.exists(save_dir_gt):
        os.makedirs(save_dir_gt)
    if not os.path.exists(save_dir_lq):
        os.makedirs(save_dir_lq)

    
        
    file_path = path
    image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)

    if image is not None:
        # Crop the image into smaller sections
        for i in range(0, image.shape[0], crop_size[1]):
            for j in range(0, image.shape[1], crop_size[0]):
                cropped_image_gt = image[i:i+crop_size[1], j:j+crop_size[0]]
                crop_index = (i // crop_size[1], j // crop_size[0])
                        
                if np.sum(cropped_image_gt == 0) == 0:  # Prüfen auf nicht-schwarze Pixel
                    # Überprüfen, ob die Kachel die volle Größe hat (für Kantenkacheln)
                    if cropped_image_gt.shape[0] == crop_size[0] and cropped_image_gt.shape[1] == crop_size[1]:

                        # Generate a unique filename for each cropped gt image
                        cropped_filename_gt = f"crop_{crop_index[0]}_{crop_index[1]}.png"
                        save_path_gt = os.path.join(save_dir_gt, cropped_filename_gt)

                        # Save the cropped gt image
                        cv2.imwrite(save_path_gt, cropped_image_gt)
                        
                        # Downsample the cropped image
                        new_size = (crop_size[0] // scale, crop_size[1] // scale)
                        cropped_image_lq = cv2.resize(cropped_image_gt, new_size, interpolation=cv2.INTER_AREA)
                        
                        # Generate a unique filename for each cropped lq image
                        cropped_filename_lq = f"crop_{crop_index[0]}_{crop_index[1]}.png"
                        save_path_lq = os.path.join(save_dir_lq, cropped_filename_lq)

                        # Save the cropped lq image
                        cv2.imwrite(save_path_lq, cropped_image_lq)
                        
path = '/home/jovyan/work/satellite_data/v2/S2A_MSIL1C_20220627T100611_N0400_R022_T33UWS_20220627T134958.tif'
load_and_crop_images(path, 3)